In [1]:
library(DESeq2)
library(tximport)
library(readr)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colMeans, colnames,
    colSums, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, lengths, Map, mapply, match,
    mget, order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rowMeans, rownames, rowSums, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which, which.max, which.min

Warning message:


Following procedures discussed in:
* http://bioconductor.org/packages/release/bioc/vignettes/tximport/inst/doc/tximport.html
* https://support.bioconductor.org/p/84059/
* http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#transcript-abundance-files-and-tximport-input
* https://dockflow.org/workflow/rnaseq-gene/#exploratory-analysis-and-visualization
* https://f1000research.com/articles/4-1521/v2
* http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#data-transformations-and-visualization

In [40]:
m=read.csv("../kallisto/sample_descriptions/samples.csv",colClasses=c("character",rep("factor",3)))
head(m)

run,sample,experiment,study
SRR787291,SAMN01942529,SRX252526,PRJNA192876
SRR787292,SAMN01942529,SRX252526,PRJNA192876
SRR787293,SAMN01942529,SRX252526,PRJNA192876
SRR787294,SAMN01942529,SRX252526,PRJNA192876
SRR787295,SAMN01942529,SRX252526,PRJNA192876
SRR790623,SAMN01942529,SRX253977,PRJNA192876


In [7]:
i=read.delim("cosmoss_V3.3.mRNA_ncRNA.tx2gene.full.tsv",sep="\t",stringsAsFactors=FALSE)
head(i)
tx2gene=i[,1:2]

TXNAME,GENEID,TYPE,Alias,support,origin,isoform,longest_transcript,has_evidence,class,has_protein_homology,has_proteomics_support,encoded_by,is_filtered
Pp3c1_20V3.1,Pp3c1_20,mRNA,"P2.evm.model.Chr01.1.6,P2.EVM%20prediction%20Chr01.1,P2.Chr01.mRNA.2,Pp3c1_20P2.2,Pp1s283_2V6.1,Pp3c1_20V1.1,Pp3c1_20P2.1",supported_by_EST_or_cDNA,NA,major,true,true,NA,true,false,protein_coding_gene,false
Pp3c1_20V3.2,Pp3c1_20,mRNA,"Phpatv32000001m,PAC4GC:5754643,Pp3c1_20J4.1",supported_by_EST_or_cDNA,NA,alternative,false,true,NA,true,false,protein_coding_gene,false
Pp3c1_40V3.2,Pp3c1_40,mRNA,"T1.asmbl_24%7Cm.6444,T1.ORF,T1.Chr01.mRNA.44,Pp3c1_40T1.12,Pp1s283_4V6.1,Pp3c1_40V1.1,Pp3c1_40T1.1",supported_by_EST_or_cDNA,NA,alternative,true,true,NA,true,false,protein_coding_gene,false
Pp3c1_40V3.1,Pp3c1_40,mRNA,"Phpatv32000002m,PAC4GC:5752909,Pp3c1_40J4.1",supported_by_EST_or_cDNA,NA,major,false,true,NA,true,false,protein_coding_gene,false
Pp3c1_50V3.2,Pp3c1_50,mRNA,"J2.PAC:28256409,J2.Pptv304000003m,J2.Chr01.mRNA.3,Pp3c1_50J2.1,Pp1s283_5V6.1,Pp3c1_50V1.1",supported_by_EST_or_cDNA,NA,alternative,false,true,NA,true,false,protein_coding_gene,false
Pp3c1_50V3.1,Pp3c1_50,mRNA,"Phpatv32000003m,PAC4GC:5753140,Pp3c1_50J4.1",supported_by_EST_or_cDNA,NA,major,true,true,NA,true,false,protein_coding_gene,false


In [22]:
table(ifelse(is.na(i$class),i$TYPE,i$class))


           28s_rRNA,rRNA_28S                antisense_RNA 
                         161                            3 
                arginyl_tRNA             asparaginyl_tRNA 
                          52                           32 
               aspartyl_tRNA                   attenuator 
                          26                            9 
              C_D_box_snoRNA                       CRISPR 
                         410                           31 
              cysteinyl_tRNA               five_prime_UTR 
                          13                            3 
             glutaminyl_tRNA                glutamyl_tRNA 
                          25                           23 
                 glycyl_tRNA               group_I_intron 
                          57                            2 
             group_II_intron             H_ACA_box_snoRNA 
                          67                            8 
               histidyl_tRNA internal_ribosome_entry_si

In [23]:
length(unique(i$GENEID))
length(unique(paste(i$GENEID,ifelse(is.na(i$class),i$TYPE,i$class),sep=".")))

[1] 38346

[1] 39246

In [24]:
tx2gene$GENEID=paste(i$GENEID,ifelse(is.na(i$class),i$TYPE,i$class),sep=".")

In [15]:
s=read.delim(gzfile("../kallisto/n_processed.tsv.gz"),sep="\t")
colnames(s)[1]="run"
head(s)

run,n_processed
1-W3,12301216
10-LG3,17790047
11-LG3-2,16522602
12-LG3-3,17634783
13-1-3-2,18778331
14-1-3-3,25190954


In [16]:
files=file.path("../kallisto/kallisto/",m$run, "abundance.h5")
head(files)

[1] "../kallisto/kallisto//SRR787291/abundance.h5"
[2] "../kallisto/kallisto//SRR787292/abundance.h5"
[3] "../kallisto/kallisto//SRR787293/abundance.h5"
[4] "../kallisto/kallisto//SRR787294/abundance.h5"
[5] "../kallisto/kallisto//SRR787295/abundance.h5"
[6] "../kallisto/kallisto//SRR790623/abundance.h5"

In [17]:
txi = tximport(files, type = "kallisto", txOut = TRUE)
head(txi$counts)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

Pp3c1_20V3.1,3.660000e+02,4.650000e+02,586.00000,402.0000,632.0000,39.00000,29.00000,16.000000,117.00000,78.00000,⋯,671.00000,485.00000,687.0000,791.00000,677.00000,5.640000e+02,692.00000,6.090000e+02,747.00000,999.00000
Pp3c1_40V3.2,8.476866e+02,1.381674e+03,1066.45214,1071.8183,777.7214,39.01141,62.78523,25.577433,170.47007,105.75214,⋯,2107.43791,1475.19079,2355.8119,2500.80916,1969.80309,1.796082e+03,2044.28578,1.856930e+03,2146.42493,2901.55653
Pp3c1_40V3.1,3.953072e+02,5.397224e+02,421.06565,488.4427,355.2439,23.94464,15.72057,10.401908,40.49952,20.98138,⋯,117.90018,50.01938,184.0701,137.54605,94.54942,6.958301e+01,69.93929,1.586308e+02,234.57971,253.41826
Pp3c1_60V3.9,0.000000e+00,0.000000e+00,47.33513,0.0000,30.2338,0.00000,24.33509,0.000000,25.52430,84.73234,⋯,234.37061,80.62298,88.8029,313.98945,251.71723,2.936190e+02,375.58716,1.720344e+02,130.45105,192.53078
Pp3c1_60V3.6,6.506609e-03,9.549311e-02,0.00000,0.0000,0.0000,0.00000,0.00000,7.852298,13.94017,0.00000,⋯,298.43651,317.97430,321.1931,164.10414,342.22705,9.718231e-03,223.47317,2.918890e+02,387.14652,230.72290
Pp3c1_60V3.1,1.538045e+02,1.715704e+02,178.93846,204.5731,148.1856,0.00000,16.96552,0.000000,35.35800,25.26766,⋯,36.85137,44.77060,190.4426,34.36286,128.21920,1.631078e-02,19.90409,2.898612e-03,69.49835,77.22188


In [25]:
gxi=summarizeToGene(txi,tx2gene)

transcripts missing from tx2gene: 2
summarizing abundance
summarizing counts
summarizing length


In [26]:
head(gxi$counts)
dim(gxi$counts)

Pp3c1_100.mRNA,1176,1151,2641,726,1591,56,50,29,173,121,⋯,3079,942,4442,3229,1457,6342,4073,893,4745,2878.0000
Pp3c1_1000.mRNA,173,273,545,195,823,182,171,83,317,245,⋯,774,601,612,718,817,920,1004,465,446,684.0000
Pp3c1_10000.mRNA,1742,1708,1272,1269,1078,207,139,128,320,342,⋯,1799,1928,1415,2488,1952,1504,2407,2405,1421,2387.0000
Pp3c1_10010.mRNA,176,221,353,160,663,163,197,100,421,267,⋯,941,600,849,890,850,851,996,526,805,956.0072
Pp3c1_10020.seryl_tRNA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0.0000
Pp3c1_10040.mRNA,405,734,824,590,856,114,119,56,224,181,⋯,709,681,787,783,1015,851,855,567,634,748.0000


[1] 37359   373

In [27]:
head(txi$abundance)

Pp3c1_20V3.1,1.280956e+01,11.243974071,16.7566488,11.208454,17.4967086,9.784241,5.501916,5.398652,11.3277470,5.869814,⋯,12.4842594,8.4542476,13.026703,14.2219895,11.671432,9.902248e+00,13.6526351,1.365712e+01,14.301298,14.5267598
Pp3c1_40V3.2,2.126995e+01,23.969079370,21.8837563,21.463815,15.4681660,6.945636,8.500405,6.134563,11.7856209,5.687575,⋯,28.0433462,18.3782187,31.949687,32.1479821,24.269382,2.256070e+01,28.8440835,2.976222e+01,29.451932,30.1691828
Pp3c1_40V3.1,4.624770e+01,43.376899995,39.9336604,44.850536,32.3229528,21.937797,10.368235,12.630693,13.5553353,5.419649,⋯,7.4815001,2.9920035,11.900944,8.4586166,5.595884,4.155611e+00,4.7089042,1.220585e+01,15.050973,12.5941678
Pp3c1_60V3.9,0.000000e+00,0.000000000,0.9203594,0.000000,0.5698565,0.000000,3.119509,0.000000,1.6709619,4.315623,⋯,2.9537778,0.9512007,1.140656,3.8226851,2.937001,3.493239e+00,5.0190498,2.611223e+00,1.695767,1.8959124
Pp3c1_60V3.6,1.472033e-04,0.001493909,0.0000000,0.000000,0.0000000,0.000000,0.000000,1.694471,0.8681079,0.000000,⋯,3.5784841,3.5689601,3.925253,1.9007661,3.798738,1.100068e-04,2.8412220,4.214863e+00,4.789294,2.1615690
Pp3c1_60V3.1,4.269612e+00,3.292268622,4.0612645,4.530125,3.2588716,0.000000,2.544361,0.000000,2.7073600,1.504735,⋯,0.5428724,0.6175914,2.859272,0.4890696,1.749234,2.267949e-04,0.3109099,5.144123e-05,1.055109,0.8889356


In [49]:
dds=DESeqDataSetFromTximport(gxi,m, ~sample)

using counts and average transcript lengths from tximport


In [50]:
colData(dds)

DataFrame with 373 rows and 4 columns
            run       sample experiment       study
    <character>     <factor>   <factor>    <factor>
1     SRR787291 SAMN01942529  SRX252526 PRJNA192876
2     SRR787292 SAMN01942529  SRX252526 PRJNA192876
3     SRR787293 SAMN01942529  SRX252526 PRJNA192876
4     SRR787294 SAMN01942529  SRX252526 PRJNA192876
5     SRR787295 SAMN01942529  SRX252526 PRJNA192876
...         ...          ...        ...         ...
369   70-LG14-2    70.LG14.2        LG3      PpDEK1
370   71-LG14-3    71.LG14.3        LG3      PpDEK1
371     72-1-14      72.1.14         OE      PpDEK1
372   73-1-14-2    73.1.14.2         OE      PpDEK1
373   74-1-14-3    74.1.14.3         OE      PpDEK1

In [51]:
estimateSizeFactors(dds)

using 'avgTxLength' from assays(dds), correcting for library size


class: DESeqDataSet 
dim: 37359 373 
metadata(1): version
assays(3): counts avgTxLength normalizationFactors
rownames(37359): Pp3c1_100.mRNA Pp3c1_1000.mRNA ... Pp3s997_10.mRNA
  Pp3s998_10.ncRNA
rowData names(0):
colnames: NULL
colData names(4): run sample experiment study

In [32]:
vsd=vst(dds, blind=FALSE)

using 'avgTxLength' from assays(dds), correcting for library size


In [33]:
head(assay(vsd), 3)

vsd

,1,2,3,4,5,6,7,8,9,10,⋯,364,365,366,367,368,369,370,371,372,373
Pp3c1_100.mRNA,9.862156,9.351662,10.623834,8.916050,9.964518,8.777317,8.434408,8.490788,8.997075,8.532412,⋯,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,7.381567,7.559691,8.580751,7.305779,9.089045,10.578629,10.377033,10.091441,9.826457,9.419218,⋯,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.427010,9.913541,9.623924,9.697720,9.450088,10.555744,9.812155,10.504621,9.839770,9.944695,⋯,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608


class: DESeqTransform 
dim: 37359 373 
metadata(1): version
assays(1): ''
rownames(37359): Pp3c1_100.mRNA Pp3c1_1000.mRNA ... Pp3s997_10.mRNA
  Pp3s998_10.ncRNA
rowData names(4): baseMean baseVar allZero dispFit
colnames(373): 1 2 ... 372 373
colData names(4): run sample experiment study

In [34]:
colData(vsd)

DataFrame with 373 rows and 4 columns
            run       sample experiment       study
    <character>     <factor>   <factor>    <factor>
1     SRR787291 SAMN01942529  SRX252526 PRJNA192876
2     SRR787292 SAMN01942529  SRX252526 PRJNA192876
3     SRR787293 SAMN01942529  SRX252526 PRJNA192876
4     SRR787294 SAMN01942529  SRX252526 PRJNA192876
5     SRR787295 SAMN01942529  SRX252526 PRJNA192876
...         ...          ...        ...         ...
369   70-LG14-2    70.LG14.2        LG3      PpDEK1
370   71-LG14-3    71.LG14.3        LG3      PpDEK1
371     72-1-14      72.1.14         OE      PpDEK1
372   73-1-14-2    73.1.14.2         OE      PpDEK1
373   74-1-14-3    74.1.14.3         OE      PpDEK1

In [35]:
e=assay(vsd)
colnames(e)=colData(vsd)$run
head(e)
write.table(e,gzfile("all_genes.vst.tsv.gz"),sep="\t",row.names=TRUE,col.names=TRUE,quote=FALSE)

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,⋯,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,9.862156,9.351662,10.623834,8.916050,9.964518,8.777317,8.434408,8.490788,8.997075,8.532412,⋯,10.256458,8.946873,10.769640,10.293939,9.185842,11.330099,10.749067,8.716056,10.795856,9.818899
Pp3c1_1000.mRNA,7.381567,7.559691,8.580751,7.305779,9.089045,10.578629,10.377033,10.091441,9.826457,9.419218,⋯,8.284943,8.206360,8.019149,8.240060,8.342942,8.548026,8.676593,7.724645,7.632214,7.874430
Pp3c1_10000.mRNA,10.427010,9.913541,9.623924,9.697720,9.450088,10.555744,9.812155,10.504621,9.839770,9.944695,⋯,9.518657,9.931283,9.188891,9.936725,9.590335,9.326010,10.018545,10.072072,9.137332,9.567608
Pp3c1_10010.mRNA,7.404392,7.265683,7.924842,7.061676,8.785212,10.227576,10.298046,10.163573,10.220884,9.600121,⋯,8.642796,8.356125,8.505761,8.541699,8.472045,8.560633,8.811026,8.035877,8.379157,8.341642
Pp3c1_10020.seryl_tRNA,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,⋯,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570,4.192570
Pp3c1_10040.mRNA,8.425789,8.747516,9.015093,8.642327,9.115491,9.740131,9.597228,9.368856,9.346674,9.067421,⋯,8.270825,8.517641,8.403065,8.371686,8.701931,8.555978,8.605236,8.127675,8.066973,8.026048


In [36]:
ee=gxi$abundance
colnames(ee)=m$run
head(ee)
write.table(ee,gzfile("all_genes.tpm.tsv.gz"),sep="\t",row.names=TRUE,col.names=TRUE,quote=FALSE)

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,⋯,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,67.266747,45.416014,123.16123,32.948485,71.65476,23.49970,15.66638,16.30608,27.62160,14.98788,⋯,94.13192,27.02578,138.39263,95.47015,41.34643,182.83480,132.06252,32.95897,148.57072,68.80491
Pp3c1_1000.mRNA,10.760345,12.254279,30.02399,9.870658,40.97173,94.24088,70.68071,57.44172,54.86589,31.44669,⋯,23.72812,16.61699,19.64246,22.62305,23.68461,26.40355,31.68660,16.36870,15.10008,17.13881
Pp3c1_10000.mRNA,57.030226,38.639119,34.03100,33.110430,27.92972,48.46456,24.64193,40.32073,28.95449,24.05729,⋯,31.29227,31.41480,25.08435,41.81832,31.45607,24.68877,44.39614,50.41413,25.44615,32.44869
Pp3c1_10010.mRNA,7.411845,6.426935,12.13748,5.360869,22.05312,49.56929,45.12251,40.85212,49.18672,24.23192,⋯,21.10285,12.61309,19.40381,19.29257,17.67286,18.00515,23.68660,14.22525,18.54847,16.75914
Pp3c1_10020.seryl_tRNA,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Pp3c1_10040.mRNA,22.263379,27.838141,36.93758,25.743655,37.06742,45.86703,35.79566,30.20142,34.33999,21.53108,⋯,20.81998,18.76333,23.55152,22.23489,27.66146,23.56678,26.62755,20.09750,19.07651,17.17564


In [37]:
eee=gxi$abundance
colnames(eee)=m$run
head(eee)
write.table(eee,gzfile("all_transcripts.tpm.tsv.gz"),sep="\t",row.names=TRUE,col.names=TRUE,quote=FALSE)

,SRR787291,SRR787292,SRR787293,SRR787294,SRR787295,SRR790623,SRR790624,SRR790625,SRR790254,SRR790259,⋯,65-D14-3,66-LO14,67-LO14-2,68-LO14-3,69-LG14,70-LG14-2,71-LG14-3,72-1-14,73-1-14-2,74-1-14-3
Pp3c1_100.mRNA,67.266747,45.416014,123.16123,32.948485,71.65476,23.49970,15.66638,16.30608,27.62160,14.98788,⋯,94.13192,27.02578,138.39263,95.47015,41.34643,182.83480,132.06252,32.95897,148.57072,68.80491
Pp3c1_1000.mRNA,10.760345,12.254279,30.02399,9.870658,40.97173,94.24088,70.68071,57.44172,54.86589,31.44669,⋯,23.72812,16.61699,19.64246,22.62305,23.68461,26.40355,31.68660,16.36870,15.10008,17.13881
Pp3c1_10000.mRNA,57.030226,38.639119,34.03100,33.110430,27.92972,48.46456,24.64193,40.32073,28.95449,24.05729,⋯,31.29227,31.41480,25.08435,41.81832,31.45607,24.68877,44.39614,50.41413,25.44615,32.44869
Pp3c1_10010.mRNA,7.411845,6.426935,12.13748,5.360869,22.05312,49.56929,45.12251,40.85212,49.18672,24.23192,⋯,21.10285,12.61309,19.40381,19.29257,17.67286,18.00515,23.68660,14.22525,18.54847,16.75914
Pp3c1_10020.seryl_tRNA,0.000000,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,⋯,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
Pp3c1_10040.mRNA,22.263379,27.838141,36.93758,25.743655,37.06742,45.86703,35.79566,30.20142,34.33999,21.53108,⋯,20.81998,18.76333,23.55152,22.23489,27.66146,23.56678,26.62755,20.09750,19.07651,17.17564
